# PSET 8: <br> Helium Atom, <br> CI, intro calculations and basis sets!

This pset will focus on three strategies of improving our wavefunctions:

* **Augmentaing our wavefunction with CIs** By building a poor man's ground wavefunction (No electron-electron repulsion) and adding in a **smart way** more excitations to the wavefunction.
* **Treating missing effect via pertubation theory**, by adding the electron-electron repulsion perturbatively to an initial approximation.
* **Using a bigger basis set of  possible atomic orbitals.** For this we will use Hartree Fock theory! The building block for all Quantum chemistry calculations.

All this to improve the accuracy for this little two proton, two electron fellow:
[![](files/Helium.png)](https://en.wikipedia.org/wiki/Helium)

Viewed is the charge density distribuiton of the helium atom.
## Import Libraries and Functions

In [ ]:
#Here is what we usually import
% matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import simps
import scipy as sp
import scipy.sparse as sparse
import scipy.sparse.linalg
import scipy.integrate as integrate
from matplotlib import animation
##this let's us print out the available functions in our library,
## as well as the documentation
import pydoc 
# add all probables spots for the quantum world library
import sys
sys.path.append('../library')
sys.path.append('../../library')
#sys.path.append('/home/student/chem160/library')
#This is how we'll import our own home-made modules
import quantumWorld as qworld

# <i class="fa fa-check-circle-o"></i> Exercise 1: <br>  Poor man's Helium + CI
## 1 point

in the pencasts and in lecture you saw that, if we make the "very poorman's" approximation for the Helium atom (i.e. neglect in the Hamiltoninan the interaction term between the two electrons) we get a system that is easy to solve (basically two independent hydrogen-like 1-electron systems) but with a crappy value for the energy (as compared to experiments).

Here we'll try something a bit more sophisticated. 

We will:

* Build a Hamiltonian matrix with a basis set of two elements: ground and excited states
* Solve the eigenvalue problem to get a the optimal mix of both solutions.
* Explore pertubation theory on this basis set.





## Part 1: Poor man's approximation
#### What follows is pretty extensive documentation of all the math steps. Please read through it. 

We'll include a second term in our expression for the two-electron wave function which will represent an electron configuration where one electron is in the 1S hydrogen-like orbital and the other is in a 2S hydrogen-like orbital:

$$\Psi(\vec{r_1},\vec{r_2}) = c_1\psi_1(\vec{r_1},\vec{r_2}) + c_2\psi_2(\vec{r_1},\vec{r_2})$$
<br>

$$\Psi(\vec{r_1},\vec{r_2}) = c_1\;\phi_{1S}(\vec{r_1})\phi_{1S}(\vec{r_2}) + c_2[\frac{1}{\sqrt{2}}(\;\phi_{1S}(\vec{r_1})\phi_{2S}(\vec{r_2})+\phi_{1S}(\vec{r_2})\phi_{2S}(\vec{r_1})\;)]\;)$$
<br>

Notice that the second term $[\phi_{1S}(\vec{r_1})\phi_{2S}(\vec{r_2})+\phi_{1S}(\vec{r_2})\phi_{2S}(\vec{r_1})]$ is symmetric with respect to exchange of the two electrons, i.e. it represents a singlet state. 



### Eigenvalues of Hydrogen-like atomic orbitals
Recall that the hydrogen-like orbitals are eigenfunctions of the operator $(-\frac{1}{2}\nabla_{r_1}^2 - \frac{Z}{r_1})$ with corresponding eigenvalues (in atomic units) that depend on $Z$, the charge of the nucleus, and the $n$ quantum number of the orbital:


$$(-\frac{1}{2}\nabla_{r_1}^2 - \frac{Z}{r_1})\phi_{1S}(\vec{r_1}) = -Z^2\frac{1}{2}\phi_{1S}(\vec{r_1})$$

$$(-\frac{1}{2}\nabla_{r_1}^2 - \frac{Z}{r_1})\phi_{2S}(\vec{r_1}) = -\frac{Z^2}{2^2}\frac{1}{2}\phi_{2S}(\vec{r_1}) = -\frac{Z^2}{8}\phi_{2S}(\vec{r_1})$$

### The Hamiltonian for the Helium atom. 

You should know by <i class="fa fa-heart"></i> that the Hamiltonian for the Helium atom is given by (in atomic units) 
![Helium atom](files/HeliumReduced.png)


$$\hat{H} = -\frac{1}{2}\nabla_{r_1}^2 -\frac{1}{2}\nabla_{r_2}^2 - \frac{2}{r_1} - \frac{2}{r_2} + \frac{1}{r_{12}}   $$

### Matrix representation of the Hamiltonian in our Configuration Interaction basis set.

We will write the matrix representation of the Hamiltonian as follows:

$H = \begin{pmatrix}
H_{11} & H_{12} \\
H_{21} & H_{22}  \end{pmatrix} $

where the different terms are given by: 


$$H_{11} = < \phi_{1S}(\vec{r_1})\phi_{1S}(\vec{r_2})|\;\hat{H}\;|\;\phi_{1S}(\vec{r_1})\phi_{1S}(\vec{r_2})>$$

$$H_{12} = \frac{1}{\sqrt{2}}<\phi_{1S}(\vec{r_1})\phi_{1S}(\vec{r_2})\;|\;\hat{H}\;|\phi_{1S}(\vec{r_1})\phi_{2S}(\vec{r_2})+\phi_{1S}(\vec{r_2})\phi_{2S}(\vec{r_1})>$$

$$H_{21} = \frac{1}{\sqrt{2}}<\phi_{1S}(\vec{r_1})\phi_{2S}(\vec{r_2})+\phi_{1S}(\vec{r_2})\phi_{2S}(\vec{r_1})|\;\hat{H}\;|\;\phi_{1S}(\vec{r_1})\phi_{1S}(\vec{r_2})>$$

$$H_{22} = \frac{1}{2}<\phi_{1S}(\vec{r_1})\phi_{2S}(\vec{r_2})+\phi_{1S}(\vec{r_2})\phi_{2S}(\vec{r_1}))|\;\hat{H}\;|\phi_{1S}(\vec{r_1})\phi_{2S}(\vec{r_2})+\phi_{1S}(\vec{r_2})\phi_{2S}(\vec{r_1})>$$

## Figuring out the different terms:

### $H_{11}$

Now, let's expand the first term to see what it is in its full glory by writing the full expression for the Hamiltonian (we did this in class when we went over the Poor Man's - not the Very Poor Man's - approximation, i.e. 1st order perturbation theory.  But I'll rederive it here to make the connection):

$$H_{11} = < \phi_{1S}(\vec{r_1})\phi_{1S}(\vec{r_2})\;|\;\hat{H}\;|\;\phi_{1S}(\vec{r_1})\phi_{1S}(\vec{r_2})>$$

$$H_{11} = < \phi_{1S}(\vec{r_1})\phi_{1S}(\vec{r_2})\;|-\frac{1}{2}\nabla_{r_1}^2 -\frac{1}{2}\nabla_{r_2}^2 - \frac{2}{r_1} - \frac{2}{r_2} + \frac{1}{r_{12}} |\;\phi_{1S}(\vec{r_1})\phi_{1S}(\vec{r_2})>$$

Let us split this up into three different terms: 

#### First term of $H_{11}$:

First a term where the hamiltonian only depends on $r1$
$$H_{11a} = < \phi_{1S}(\vec{r_1})\phi_{1S}(\vec{r_2})\;|-\frac{1}{2}\nabla_{r_1}^2 - \frac{2}{r_1}|\;\phi_{1S}(\vec{r_1})\phi_{1S}(\vec{r_2})>$$

As we've pointed out in class and in the pencasts, you should notice the fact that the operator that appears in this expression ($-\frac{1}{2}\nabla_{r_1}^2 - \frac{2}{r_1}$) is pretty much the Hamiltonian for the Hydrogen atom in atomic units. And since $\phi_{1S}$ is an eigenstate of this Hamiltonian: 

$$(-\frac{1}{2}\nabla_{r_1}^2 - \frac{2}{r_1})\phi_{1S}(\vec{r_1}) = -Z^2\frac{1}{2}\phi_{1S}(\vec{r_1})$$ 

and $\phi_{1S}$ is normalized, you should be able to see this expression is equal to:

$$H_{11a} = -(Z=2)^2\frac{1}{2} = -2$$

#### Second term of $H_{11}$:
In the next term, the Hamiltonian only depends on $r2$
$$H_{11b} = < \phi_{1S}(\vec{r_1})\phi_{1S}(\vec{r_2})\;| -\frac{1}{2}\nabla_{r_2}^2 - \frac{2}{r_2} |\;\phi_{1S}(\vec{r_1})\phi_{1S}(\vec{r_2})>$$
<br> 
by the same arguments as above yield that:

$$H_{11b} = -2$$

#### Third term of $H_{11}$:

The next term is more tricky, with its $\frac{1}{r_{12}}$ term. 
<br>
$$H_{11c} = < \phi_{1S}(\vec{r_1})\phi_{1S}(\vec{r_2})\;| \frac{1}{r_{12}} |\;\phi_{1S}(\vec{r_1})\phi_{1S}(\vec{r_2})>$$
<br> 
To solve it, you need some funky integration techniques.  But it turns out the integral equals:

$$H_{11c} = \frac{5}{4}$$

Putting these three terms together, we have:
$$H_{11} = -2 -2 + \frac{5}{4} = -\frac{11}{4}$$

In [ ]:
H_11 = -2 - 2 + 5.0/4.0
# Hard science right here
print H_11

## $H_{12} = H_{21}$
The second term looks a bit more intimidating, but it turns out that most of the integrals go to zero because of the orthogonality of $\phi_{1S}(\vec{r})$ and $\phi_{2S}(\vec{r})$ 


$$H_{12} = <\phi_{1S}(\vec{r_1})\phi_{1S}(\vec{r_2})\;|\;\hat{H}\;|\;\phi_{1S}(\vec{r_1})\phi_{2S}(\vec{r_2})+\phi_{1S}(\vec{r_2})\phi_{2S}(\vec{r_1})>$$

Here is $H_{12}$ with the Hamiltonian fully expanded:

$$H_{12} = <\phi_{1S}(\vec{r_1})\phi_{1S}(\vec{r_2})\;|-\frac{1}{2}\nabla_{r_1}^2 -\frac{1}{2}\nabla_{r_2}^2 - \frac{2}{r_1} - \frac{2}{r_2} + \frac{1}{r_{12}}|\;\phi_{1S}(\vec{r_1})\phi_{2S}(\vec{r_2})+\phi_{1S}(\vec{r_2})\phi_{2S}(\vec{r_1})>$$

We can split this up into two different terms. And it turns out (again because of orthogonality of the hydrogen-like atom eigenstates) that these expressions reduce to the following integrals:

$$H_{12a} = <\phi_{1S}(\vec{r_1})\phi_{1S}(\vec{r_2})\;|\;\hat{H}\;|\;\phi_{1S}(\vec{r_1})\phi_{2S}(\vec{r_2})> \;\; = <\phi_{1S}(\vec{r_1})\phi_{1S}(\vec{r_2})\;|\;\frac{1}{r_{12}}\;|\;\phi_{1S}(\vec{r_1})\phi_{2S}(\vec{r_2})>  $$
<br>

$$H_{12b} = <\phi_{1S}(\vec{r_1})\phi_{1S}(\vec{r_2})\;|\;\hat{H}\;|\;\phi_{1S}(\vec{r_2})\phi_{2S}(\vec{r_1})>\;\; = <\phi_{1S}(\vec{r_1})\phi_{1S}(\vec{r_2})\;|\;\frac{1}{r_{12}}\;|\;\phi_{1S}(\vec{r_2})\phi_{2S}(\vec{r_1})>\ $$



In [ ]:
H_12a = (4096.0/64827.0)*np.sqrt(2)*2  # Integrals, precomputed
H_12b = (4096.0/64827.0)*np.sqrt(2)*2  # Integrals, precomputed
H_12 = (1.0/np.sqrt(2))*(H_12a + H_12b)
H_21 = H_12
print H_12

# $H_{22}$

The last term looks even more intimidating, but it can be dealt with:

$$H_{22} = <\phi_{1S}(\vec{r_1})\phi_{2S}(\vec{r_2})+\phi_{1S}(\vec{r_2})\phi_{2S}(\vec{r_1})|\;\hat{H}\;|\phi_{1S}(\vec{r_1})\phi_{2S}(\vec{r_2})+\phi_{1S}(\vec{r_2})\phi_{2S}(\vec{r_1})>$$

We can expand this out into four terms. And one can show that:

$$H_{22} = \frac{1}{2} (2\;J_{22,22} + 2\;K_{22,22})$$

Where

$$J_{22} = <\phi_{1S}(\vec{r_1})\phi_{2S}(\vec{r_2})|\;\hat{H}\;|\phi_{1S}(\vec{r_1})\phi_{2S}(\vec{r_2})> \;\; =  -2 - \frac{1}{2} + \;\; <\phi_{1S}(\vec{r_1})\phi_{2S}(\vec{r_2})|\;\frac{1}{r_{12}}\;|\phi_{1S}(\vec{r_1})\phi_{2S}(\vec{r_2})> $$ 

$$K_{22} = <\phi_{1S}(\vec{r_1})\phi_{2S}(\vec{r_2})|\;\hat{H}\;|\phi_{1S}(\vec{r_2})\phi_{2S}(\vec{r_1})>\;\; = \;\;<\phi_{1S}(\vec{r_1})\phi_{2S}(\vec{r_2})|\;\frac{1}{r_{12}}\;|\phi_{1S}(\vec{r_2})\phi_{2S}(\vec{r_1})> $$

We'll give you the expressions for these below:


In [ ]:

J_22 = -5.0/2.0 + (17.0/81.0)*2 
K_22 = (16.0/729.0)*2
# fill me here
H_22 = 


##  Build $H = \begin{pmatrix} H_{11} & H_{12} \\ H_{21} & H_{22}  \end{pmatrix} $

Putting all this together, we can now build the matrix  

In [ ]:
H = np.zeros((2,2))


#######


### Get the eigenvalues of the $H$ matrix

Print the ground state energy and look at the $c1$ and $c2$ coeficicents

In [ ]:


print 'The ground state energy estimate is --> ', eigvals[0]
print 'The c1 and c2 coefficients are --> ', eigvecs[:,0] 

## <i class="fa fa-question-circle"></i> Compare

How does this estimate compare to:

1.  The exact ground state energy of He?

2.  The Very Poor Man's approximation to the ground state energy of He?

3.  The Poor Man's approximation?

Recall that here, we are working in the atomic unit of energy, the Hartree.  Whereas in the lecture notes, sometimes the energies are given in electron volts.  You might need the following conversion factor:


In [ ]:
Hartree_to_Ev = 27.211399

print 'Exact Energy --> -79 eV'
print 'Very Poor Mans approx. --> -108.8 eV'
print 'Poor Mans approx (1st Perturbation Theory using Very poor mans orbitals) --> -74.8 eV'
print 'Our Configuration interaction estimate --> ', eigvals[0]*Hartree_to_Ev

## Part 2: First Order Pertubation theory 

Pertubation theory can be used to "recover" what is lost when using approximations, it works by assuming that your exact solution can be described as a power series, for example if $E_{Exact}$ is the exact energy , this would be:
$$
E_{Exact} = E_{0} + \lambda E_{1} + \lambda^2 E_{2} + \lambda^3 E_{3} \dots
$$

where $E_0$ is a first approximation to the problem, $E_i$ are smaller higher order corrections and $\lambda$ is the pertubation parameter.
We can apply this idea to first order on the Hamiltonian:

$$H = H_{0} + \lambda H_{1} $$

Where $H_{0}$ is our Very Poor Man's hamiltonian approximation and the term $H_{1} = \frac{1}{r_{12}}$, will treat the interaction between the two electrons, i.e. as a pertubation. In this case $\lambda$ will be a knob that we can gradually "turn on and off" by multiplying the interaction term, and make it change from 0 to 1.
When $\lambda = 0$ is Very Poor Man's aproximation.  

When $\lambda = 1$  we are back with the full Coulomb interaction.  And in between we are somewhere between the two scenarios. When we do this multiplication ( $\lambda\frac{1}{r_{12}}$ ), all the integrals that have  $\frac{1}{r_{12}}$ sandwiched in between will end up having a $\lambda$ factor as well.

Normally in pertubation theory you also optimize your $\lambda$ based on a certain criteria, in quantum mechanics you use the variational principle. You will not have to do this.

Your mission is to create a hamiltonian that depends on lambda:

In [ ]:
#Matrix for different values of lambda. 

def H_lambda(Lambda):
    
    H = np.zeros((2,2))
        
    return H

### Vary $\lambda$ from 0 to 1, and get the ground state energy and the c1 and c2 coefficients for each value. 

In [ ]:
Lambda_array = np.arange(0,1,0.01)
c1 = np.zeros(len(Lambda_array))
c2 = np.zeros(len(Lambda_array))
Eg = np.zeros(len(Lambda_array))

###################
###Write a for loop that loops over the indices i of the
## Lambda_array array.  At each value, 
## 1. get the H matrix, 
## 2. get the c1 and c2 coefficients. Plug them in the appropriate arrays 
## 3. get the ground state energy. Plug it in the appropriate array.



###################

### <i class="fa fa-line-chart"></i> Plot both coefficients squared $c_1^2$ and $c_2^2$ as a function of $\lambda$. 

In [ ]:
plt.plot(Lambda_array, c1**2, lw=4)
plt.plot(Lambda_array, c2**2, lw = 4)
plt.legend(('$\psi_0$ (ground)','$\psi_1$ (singlet excited)'))
plt.ylim((0, 1.5))
plt.xlabel('$\lambda$ parameter (electron interaction strength)')
plt.ylabel('$c_1$ and $c_2$ coefficients')
plt.title('Contributions from ground and singlet excited vs. $\lambda$')
plt.show()

### <i class="fa fa-line-chart"></i> Plot the ground energy as a function of $\lambda$

In [ ]:
plt.axhline(-79,label="$E_{Exact}$")
plt.xlabel('$\lambda$ parameter (electron interaction strength)')
plt.ylabel('Ground State energy')
plt.title('CI ground state energy estimate vs. $\lambda$')
plt.legend(loc='best')
plt.show()

### <i class="fa fa-question-circle"></i> Questions/Discussion

* Think about the plots and how to interpret them.
* Can you see how, as you graduallly turn on the interation, the energy gets closer to the experimental value, and you get more contribution from the singlet excited state?
* How would you choose the ideal $\lambda$? (Imagine you did not have the exact energy, since this is what we ultimately want)

Answer here

# <i class="fa fa-check-circle-o"></i> Exercise 2: <br>  Hartree-Fock on Helium
## 1 point


In the previous exercise we used solutions to the hamiltonian (ground states, exicted states) as our basis set. Each solution was build from fixed $1s$ and $2s$ hydrogen orbitals. Here we will explore the idea of impoving on our guess not from CI, but by increasing the number of available orbitals to choose from. Our basis set will be the atomic orbitals!

Unlike Considerable Exercise No. 2, where you will go further and code some of the detail of the Hartree-Fock method, here we'll use Hartree-Fock as implemented in PyQuante more like a **black box method**.  

![fake pyquante](files/PyQuante.jpg)
From [pyquante's website](http://pyquante.sourceforge.net/):
>PyQuante is an open-source suite of programs for developing quantum chemistry methods.
> The goal of this software is not necessarily to provide a working quantum chemistry program
>, but rather to provide a well-engineered set of tools so that scientists can construct 
> their own quantum chemistry programs without going through the tedium of having to write 
> every low-level routine.

**Note:** Some people like to prenounciate it as *picante* (spicy in spanish).

The steps for a calculation will be:

* Specify the helium via **Molecule(name, atomlist)**, check the [documentation](http://pyquante.sourceforge.net/#specifying-a-molecule) on how to do this.
* Use the **rhf(molecule, basis)** function, that takes as input a molecule and a specified basis set. We will provide the basis set string.
* Each calculation will provide you with energy values, orbital energies and orbital coeficients. We just need the energies right now. Check [if in doubt.](http://pyquante.sourceforge.net/#using-the-code)

Finally we will run multiple calculations on increasingly bigger basis sets and compare these values on a plot.



**NOTE:** In particular we will use the **rhf**, restricted hartree fock, restricted as in a closed shell...don't worry, this just means the plain vanilla version of hartree fock.


## Define the helium atom

In [ ]:
from PyQuante.Molecule import Molecule



## Run a single hartree fock calculation

The basis is not important right now.

What is the energy? 

Remember to convert from Hartree to Electron Volts if you want to compare.

In [ ]:
from PyQuante.hartree_fock import rhf



## Check one basis set

Using the **getbasis(molecule,basis_set_name)** function we will be able to retrieve information on the basis set. We currently only interested in the number of basis functions used. You can retrieve this information using len on the returned value of getbasis.

And you can even get further information with **qworld.print_orbitalInfo(molecule,basis_set_name)**.
Which will print out:

* Number of basis functions
    * For each atom, the number of atomic orbitals available to it
        * For each atomic orbital, the radial part is decomposed in terms of gaussians since these are easier to calculate.
            The general formula for each gaussian  is $ N x^a y^b z^c e^{coef * r^2}$, $N$ being a normalization constant, the $exponent=(a,b,c)$ depending on the direction and $r^2=x^2 +y^2 +z^2$.



In [ ]:
from PyQuante.Ints import getbasis
# choose one
basis_sets=['sto3g','cc-pvdz','6-311g++(2d,2p)','6-311g++(3d,3p)','cc-pvtz']
# get the information

## Effect of basis size
Run a for loop over multiple basis sets of increasing size, save the number of basis sets and the calculated energy.

Don't worry too much on what the strings mean right now, for more info check [here](https://en.wikipedia.org/wiki/Basis_set_(chemistry).

In [ ]:
energies =[]
nbfns=[]
for bset in basis_sets:
    # code for calculating and saving values


#convert to numpy array
energies  = np.array(energies)
nbfns = np.array(nbfns)

## <i class="fa fa-line-chart"></i> Plot and compare your energies

In [ ]:


plt.xlabel("Number of basis functions")
plt.ylabel("Energy ($E_v$)")
plt.title("Comparison of multiple energy calculations")
plt.show()

### <i class="fa fa-question-circle"></i> Questions/Discussion

* What trend do you see?
* What would be the tradeoffs?
* Can you get to the exact energy increasing the basis set?
* Which strategy gave better results?
* How could further improve the accuracy of your calculations?

Answer here